El dataset PadChest que vamos a usar se descarga desde https://bimcv.cipf.es/bimcv-projects/padchest/ cualquier persona lo puede usar pero se han de aceptar las condiciones. Para facilitar la descarga y organización de las imágenes dejo a continuación las funciones usadas.

Una vez se hayan aceptado las condiciones y descargado todos los archivos comprimidos, guardaremos todas las imagenes en la carpeta 'extracted_files' y organizaremos las imagenes por clases con los nombres 'normal' y 'lung_metastasis' con las imagenes correspondientes

In [ ]:
# Creamos una carpeta para cada clase
lung_metastasis_dir = os.path.join(base_path, 'lung_metastasis')
normal_dir = os.path.join(base_path, 'normal')

# Aseguramos que dichas carpetas existen
os.makedirs(lung_metastasis_dir, exist_ok=True)
os.makedirs(normal_dir, exist_ok=True)

# Función para procesar cada archivo ZIP
def process_zip_file(zip_name):
    zip_path = os.path.join(zip_folder, f'{zip_name}.zip')
    extract_dir = os.path.join(base_path, 'extracted_files')

    # Extraer el ZIP
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
        print(f"Archivos extraídos en el directorio: {extract_dir}")

    # Filtro para seleccionar las imagenes que son normales o con metastasis.
    df_filtrado = df[df['Labels'].apply(lambda x: 'lung metastasis' in x or 'normal' in x if pd.notna(x) else False)]

    # Mover imágenes filtradas a las carpetas correspondientes
    for index, row in df_filtrado.iterrows():
        source_path = os.path.join(extract_dir, row['ImageID'])
        if 'lung metastasis' in row['Labels']:
            dest_path = os.path.join(lung_metastasis_dir, row['ImageID'])
        elif 'normal' in row['Labels']:
            dest_path = os.path.join(normal_dir, row['ImageID'])

        if os.path.exists(source_path):
            shutil.move(source_path, dest_path)
        else:
            print(f"Archivo no encontrado: {source_path}")

# Iterar sobre cada archivo ZIP en la lista
for zip_name in zip_names:
    process_zip_file(zip_name)


In [ ]:
# Ejemplo de alamcenamiento de las imagenes desde los archivos comprimidos

#zip_names = ['1','2','3']
#Ruta donde se almacenarán las imagenes
#base_path = 'D:\\Datos Usuario\\Desktop\\data\\Año 2.2\\TFM\\Imagenes'

#Ruta donde se guardan los archivos comprimidos de donde se van a extraer las imagenes
#zip_folder = 'D:\\Datos Usuario\\Desktop\\data\\Año 2.2\\TFM\\Imagenes'

Una vez descargadas las almacenamos en un directorio para separarlas en dos conjuntos, uno de entrenamiento y otro de validacion

In [ ]:
# Crear los directorios si no existen
os.makedirs('data/normales', exist_ok=True)
os.makedirs('data/metastasis', exist_ok=True)

# Mover imágenes a la nueva estructura
for filename in os.listdir('normal_redimensionadas'):
    shutil.move(f'normal_redimensionadas/{filename}', 'data/normales/')

for filename in os.listdir('metastasis_augmented'):
    shutil.move(f'metastasis_augmented/{filename}', 'data/metastasis/')

In [ ]:
# Rutas a las carpetas originales
dir_normales = 'data/normales'
dir_metastasis = 'data/metastasis'

# Rutas a las carpetas de destino
train_normales = 'data/train/normales'
train_metastasis = 'data/train/metastasis'
val_normales = 'data/val/normales'
val_metastasis = 'data/val/metastasis'

# Crear las carpetas si no existen
os.makedirs(train_normales, exist_ok=True)
os.makedirs(train_metastasis, exist_ok=True)
os.makedirs(val_normales, exist_ok=True)
os.makedirs(val_metastasis, exist_ok=True)



In [ ]:
def split_and_copy_data(source, train_dir, val_dir, test_size=0.2):
    # Lista todos los archivos en la carpeta fuente
    files = [f for f in os.listdir(source) if os.path.isfile(os.path.join(source, f))]

    # Divide los archivos en entrenamiento y validación
    train_files, val_files = train_test_split(files, test_size=test_size, random_state=42)

    # Copia los archivos a las carpetas de entrenamiento y validación
    for f in train_files:
        shutil.copy(os.path.join(source, f), os.path.join(train_dir, f))
    for f in val_files:
        shutil.copy(os.path.join(source, f), os.path.join(val_dir, f))

# Aplicar la función a las carpetas de normales y metástasis
#split_and_copy_data(dir_normales, train_normales, val_normales)
#split_and_copy_data(dir_metastasis, train_metastasis, val_metastasis)


Comprobamos que la division se ha realizado de forma correcta. Esta funcion realiza el recuento por clase del numero de imagenes en los conjuntos train y val

In [ ]:
def count_images(directory):
    categories = os.listdir(directory)
    counts = {}
    for category in categories:
        path = os.path.join(directory, category)
        if os.path.isdir(path):  # Asegura que solo cuenta directorios (clases)
            counts[category] = len(os.listdir(path))
    return counts

# Conteo de imágenes en el directorio de entrenamiento y validación
train_counts = count_images('data/train')
val_counts = count_images('data/val')

print("Entrenamiento:", train_counts)
print("Validación:", val_counts)


In [ ]:
def augment_images(source_directory, target_directory):
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)

    image_count = 0

    for filename in os.listdir(source_directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(source_directory, filename)
            image = cv2.imread(image_path)
            if image is not None:
                # Lista de imágenes para incluir la original y sus transformaciones
                images = [image]

                # Aplicar rotación de 90 grados
                rotated_90 = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
                images.append(rotated_90)

                # Aplicar rotación de 180 grados
                rotated_180 = cv2.rotate(image, cv2.ROTATE_180)
                images.append(rotated_180)

                # Aplicar rotación de 270 grados
                rotated_270 = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
                images.append(rotated_270)

                # Aplicar volteo horizontal
                flipped_horizontal = cv2.flip(image, 1)
                images.append(flipped_horizontal)

                # Aplicar volteo vertical
                flipped_vertical = cv2.flip(image, 0)
                images.append(flipped_vertical)

                # Guardar todas las imágenes transformadas
                for img in images:
                    output_filename = f"{filename.split('.')[0]}_aug_{image_count}.{filename.split('.')[-1]}"
                    output_path = os.path.join(target_directory, output_filename)
                    cv2.imwrite(output_path, img)
                    image_count += 1
    print(f"Total de imágenes creadas: {image_count}")
    
# Directorio origen
# source_directory = "Raw/train/metastasis"
# Directorio destino
# target_directory = "Raw/train/metastasis_augmented"

# augment_images(source_directory, target_directory)